In [ ]:
from Tools import *
from photoObject import *
import numpy as np
import seaborn as sns
import pandas as pd
from BayesianInference import *
from random import Random
import random
from scipy.interpolate import interp1d
from scipy.stats import rv_continuous

## Load Data

In [ ]:
cat=ReadCatalogs("../Catalogs/Binospec-Candels.cat")
Photometry = [PhotoObject(c) for c in cat ]
filterNames     =   ['KPNO_U_FLUX', 'LBC_U_FLUX', 'ACS_F435W_FLUX', 'ACS_F606W_FLUX', 'ACS_F775W_FLUX', 'ACS_F814W_FLUX', 'ACS_F850LP_FLUX', 'WFC3_F105W_FLUX', 'WFC3_F125W_FLUX', 'WFC3_F140W_FLUX', 'WFC3_F160W_FLUX']
filtErrNames    =   [ 'KPNO_U_FLUXERR', 'LBC_U_FLUXERR', 'ACS_F435W_FLUXERR', 'ACS_F606W_FLUXERR', 'ACS_F775W_FLUXERR', 'ACS_F814W_FLUXERR', 'ACS_F850LP_FLUXERR', 'WFC3_F105W_FLUXERR', 'WFC3_F125W_FLUXERR', 'WFC3_F140W_FLUXERR', 'WFC3_F160W_FLUXERR']
centralWavelengths  = [3584.07,3579.29,4356.59,6000.37,7702.41,8196.29,9193.55,10651.00,12576.18,14061.91,15436.30]
EffectiveWidth      = [592.73,479.17,776.58,1871.31,1299.92,1933.96,1549.7,2371.97,2674.40,3569.86,2750.15]
maskLAE         = np.array([True if p.type=="LAE"  else False for p in Photometry])
UselessVariable = [gal.setFilters(filterNames,filtErrNames) for gal in Photometry]
UselessVariable = [gal.setCWave(centralWavelengths) for gal in Photometry]
UselessVariable = [gal.setEffWidth(EffectiveWidth) for gal in Photometry]




slopes  =  np.array([p.giveCat()["slope"] for p in Photometry]) #np.array([p.calculateUVslope(ShowPlots=False) for p in Photometry])
Muv     =   np.array([p.giveCat()["Muv"] for p in Photometry])#np.array([p.calculateMUV() for p in Photometry])
dMuv  =  np.array([p.giveCat()["dMuv"] for p in Photometry]) #np.array([p.MuvErr for p in Photometry])
zs  =   np.array([p.redshift for p in Photometry])
FWHM    =   np.array([p.giveCat()["LyaFWHM"] for p in Photometry])
LyaLum  =   np.array([p.calculateLumLya() for p in Photometry])
Skewness    =   np.array([p.giveCat()["Skewness"] for p in Photometry])
EW      = np.array([p.giveCat()["EWLya"] for p in Photometry])  #np.array([p.calculateEW() for p in Photometry])
dEW=np.array([p.giveCat()["dEWLya"] for p in Photometry])#np.array([p.getEWError() for p in Photometry])
dSlp=np.array([p.giveCat()["dslope"] for p in Photometry])#np.array([p.UVfitErrs[0] for p in Photometry])
dMuv=np.array([p.giveCat()["dMuv"] for p in Photometry])#np.array([p.MuvErr for p in Photometry])



In [ ]:
def duplicate(data):
    return np.concatenate((data,data))

mask=[]
# Objects with all the individual data
for s,m,e in zip(slopes,Muv,EW):
    if -22<m<-18 and np.isnan(e)==False:
    #if m<-18 and np.isnan(e)==False:
        mask.append(True)
    else:
        mask.append(False)


BGal = np.array([BayesGalaxy(c) for c in cat ])
for i in range(0,len(BGal)):
    BGal[i].Lum    =   LyaLum[i]
    BGal[i].zs    =   zs[i]
    BGal[i].EW    =   EW[i]
    BGal[i].dEW    =   dEW[i]
    BGal[i].UVslope    =   slopes[i]
    BGal[i].Muv    =   Muv[i]
    BGal[i].FWHM    =   FWHM[i]
    BGal[i].Skew    =   Skewness[i]
    BGal[i].dMuv    =   dMuv[i]
    BGal[i].dSlp    =   dSlp[i]


#BGal=duplicate(BGal)
#mask=duplicate(mask)

#BGal=duplicate(BGal)
#mask=duplicate(mask)
#BGal=duplicate(BGal)
#mask=duplicate(mask)
#BGal=duplicate(BGal)
#mask=duplicate(mask)
#BGal=duplicate(BGal)
#mask=duplicate(mask)

#Object with the global data inside

BInf=BayesInf()
#BInf.types  =   [g.type for g in BGal[mask]]
BInf.Lum    =   np.array([g.Lum for g in BGal[mask]])
BInf.EW    =   np.array([g.EW for g in BGal[mask]])
BInf.dEW    = np.array([g.dEW for g in BGal[mask]])
BInf.UVslope    =   np.array([g.UVslope for g in BGal[mask]])
BInf.Muv    =   np.array([g.Muv for g in BGal[mask]])
BInf.FWHM    =   np.array([g.FWHM for g in BGal[mask]])
BInf.Skew    =   np.array([g.Skew for g in BGal[mask]])
BInf.dMuv   =   np.array([g.dMuv for g in BGal[mask]])
BInf.dSlp   =   np.array([g.dSlp for g in BGal[mask]])
BInf.zs   =   np.array([g.zs for g in BGal[mask]])
BInf.orisize   =   np.array([0 for g in BGal[mask]])





## Mock data

In [ ]:
def RandomSampler(a,w):

    RAND=np.random.uniform(low=0, high=1,size=1)
    
    if RAND[0] <= (1-a):
        return 0.0

    def OriginalLikelihood(ew,A=a,Wo=w):
        return ((A/Wo)*np.exp(-ew/Wo)*np.heaviside(ew,0.0))

    class CustomDistribution(rv_continuous):
        def _pdf(self, x):
            # Replace 'funcion' with your actual PDF function
            return OriginalLikelihood(x)

    bmax=1000
    random_values=bmax
    while random_values>=bmax-10:
        custom_dist = CustomDistribution(a=0, b=bmax, name='custom_dist')
        custom_dist._pdf = np.vectorize(OriginalLikelihood)
        random_values = custom_dist.rvs(size=1)
    
    return random_values[0]



In [ ]:
def log_prior(theta,BInf,physParams=BInf.Muv):
    # I can put priors here or in the equation in the BInf object 

    Auv,Ac,Wuv,Wc= theta
    A =   BInf.ParameterModel(physParams,[Auv,Ac])
    W =   BInf.ParameterModel(physParams,[Wuv,Wc])
    #print(A,W)
    if (A >= 0.).all() and (A <=1.0).all() and (W > 0.).all():# and (W < 500.).all():
        return 0.0 

    return -np.inf


def log_likelihood(theta,y,yerr,BInf,physParams=BInf.Muv):
    Auv,Ac,Wuv,Wc = theta
    model = BInf.Posterior(y,yerr,physParams,[Auv,Ac],[Wuv,Wc]) # Does thus have to be exp, so it gets outside the logaritm?

    return np.sum(np.log(model))

    

def log_probability(theta, y, yerr,BInf):
    #DrawnMuv=np.array([np.random.normal(loc=x,scale=dx) for x,dx in zip(BInf.Muv,BInf.dMuv)])
    #DrawnSlp=np.array([np.random.normal(loc=x,scale=dx) for x,dx in zip(BInf.UVslope,BInf.dSlp)])
    DrawnphysParams=BInf.Muv
    
    lp = log_prior(theta,BInf,physParams=DrawnphysParams)
    if not np.isfinite(lp):
        return -np.inf
    
    lL=log_likelihood(theta, y, yerr,BInf,physParams=DrawnphysParams)
    if np.isnan(lL)==True:
        return -np.inf
    else:
        return lp + lL


## Test Mock Data

In [ ]:
import emcee
import pathos.multiprocessing as multiproc
from multiprocessing import Pool
import corner
from IPython.display import display, Math

%load_ext autoreload
%autoreload
plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams.update({'font.size': 22})
cm = plt.cm.get_cmap('cool_r')


mockParamsA=[0.12,3.1]
mockParamsW=[8,230]
#print(mockParamsA[0]*np.nanmedian(BInf.Muv)+mockParamsA[1]*np.nanmedian(BInf.UVslope)+mockParamsA[2])
#print(mockParamsW[0]*np.nanmedian(BInf.Muv) + mockParamsW[1]*np.nanmedian(BInf.UVslope) +mockParamsW[2])

dEWGrid=np.array([10])
SNCutGrid=np.array([0.1,1,2,3,4,5])
attempts=[1,2,3,4,5,6,7,8,9,10]

for EWerr in dEWGrid:
    for SNCUT in SNCutGrid:
        for aaa in attempts:

            x,probSets,As,Ws=BInf.GenerateMockData(mockParamsA,mockParamsW)

            mockEW=[]
            i=0
            mockA,mockW=[],[]
            for prob,a,w,m,s in zip(probSets,As,Ws,BInf.Muv,BInf.UVslope):
                ew=RandomSampler(a,w)
                if a<0 or a>1:
                    print(a,ew,m,s)
                mockEW.append(ew)
                mockA.append(a)
                mockW.append(w)

                if i%100==0:
                    print(i)
                i=i+1


            BInf.EW=np.array(mockEW)
            BInf.RandomDrawEW(noise=EWerr)
            BInf.Classify(SNcut=SNCUT)
            BInf.GenerateWtab()

            steps=5000
            nwalkers=20
            #mockParamsA=[0.02,-0.1,1.1]
            #mockParamsW=[8,-5,235]

            inValues=[0.005,1.,6,220]
            pos = inValues+ [0.01,0.2,1,30] * np.random.randn(nwalkers,len(inValues) )*0.2
            nwalkers, ndim = np.shape(pos)

            mp_pool = multiproc.ProcessPool(nodes=8)
            with mp_pool as pool:
                sampler = emcee.EnsembleSampler(
                    nwalkers, ndim, log_probability, args=(BInf.EW_obs , BInf.dEW , BInf),pool=pool
                )
                sampler.run_mcmc(pos, steps, progress=True)



            dis=2000
            labels = ["Auv","Ac","Wuv","Wc"]
            #fig, axes = plt.subplots(len(labels), figsize=(10, 7), sharex=True)
            samples = sampler.get_chain(discard=dis)
            logProb=sampler.get_log_prob(discard=dis)
            maskWalkers=[False if np.isinf(logProb[steps-dis-10,i]) else True for i in range(0,nwalkers)]
            samples=np.array([s[maskWalkers] for s in samples])



            ##########
            #Get corner plot
            ##########
            ndim=len(labels)
            samples = sampler.get_chain(discard=dis)
            samples=np.array([s[maskWalkers] for s in samples])
            samples=samples.reshape(np.shape(samples)[0]*np.shape(samples)[1],np.shape(samples)[2])
            print(np.shape(samples))


            fig = corner.corner(
                samples, labels=labels,quantiles=[.16,.50,.84],truths=[*mockParamsA,*mockParamsW]
            )
            plt.savefig("Figs/corner_dEW-"+str(EWerr)+"_snCut-"+str(SNCUT)+"_attempt-"+str(aaa)+".pdf")
            plt.clf()


            ###########
            #Get Results
            results=[]
            r16,r84=[],[]
            for i in range(ndim):
                mcmc = np.percentile(samples[:, i], [16, 50, 84])
                results.append(mcmc[1])
                r16.append(mcmc[0])
                r84.append(mcmc[2])

            ############



            mParamsA=results[:2]
            mParamsW=results[2:]

            x,probSets,As,Ws=BInf.GenerateMockDataConvolved(mockParamsA,mockParamsW)
            TrueAs=As
            TrueWs=Ws

            x,probSets,As,Ws=BInf.GenerateMockDataConvolved(mParamsA,mParamsW)

            resultAs=As
            resultWs=Ws
            mParamsA=r16[:2]
            mParamsW=r16[2:]
            x,probSets,As,Ws=BInf.GenerateMockDataConvolved(mParamsA,mParamsW)
            r16As=As
            r16Ws=Ws
            mParamsA=r84[:2]
            mParamsW=r84[2:]
            x,probSets,As,Ws=BInf.GenerateMockDataConvolved(mParamsA,mParamsW)
            r84As=As
            r84Ws=Ws

            
            All=[[TrueAs,TrueWs],[r16As,resultAs,r84As],[r16Ws,resultWs,r84Ws]]
            np.save("data/Data_dEW-"+str(EWerr)+"_snCut-"+str(SNCUT)+"_attempt-"+str(aaa)+".npy",All)

            ##Plot As
            up,down=[],[]
            for r,e in zip(r16As,r84As):
                if r>e:
                    up.append(r)
                    down.append(e)
                if e>r:
                    up.append(e)
                    down.append(r)

            sc=plt.scatter(TrueAs,resultAs,c=BInf.Muv, vmin=-23, vmax=-19, cmap=cm)
            plt.fill_between(np.sort(TrueAs),np.sort(up),np.sort(down),alpha=0.3)
            #plt.fill_between(TrueAs,r16As,r84As)
            #plt.scatter(TrueAs,r84As)
            plt.colorbar(sc,label="Muv")
            plt.plot([min(TrueAs),max(TrueAs)],[min(TrueAs),max(TrueAs)])
            plt.xlabel("True A")
            plt.ylabel("Resulting A")
            plt.savefig("Figs/As_dEW-"+str(EWerr)+"_snCut-"+str(SNCUT)+"_attempt-"+str(aaa)+".pdf")
            plt.clf()


            ##Plot Ws
            up,down=[],[]
            for r,e in zip(r16Ws,r84Ws):
                if r>e:
                    up.append(r)
                    down.append(e)
                if e>r:
                    up.append(e)
                    down.append(r)


            sc=plt.scatter(TrueWs,resultWs,c=BInf.Muv, vmin=-23, vmax=-19, cmap=cm)
            plt.fill_between(np.sort(TrueWs),np.sort(up),np.sort(down),alpha=0.3)
            plt.colorbar(sc,label="Muv")
            plt.plot([min(TrueWs),max(TrueWs)],[min(TrueWs),max(TrueWs)])
            plt.xlabel("True W0")
            plt.ylabel("Resulting W0")
            plt.savefig("Figs/Ws_dEW-"+str(EWerr)+"_snCut-"+str(SNCUT)+"_attempt-"+str(aaa)+".pdf")
            plt.clf()

In [ ]:


#cm = plt.cm.get_cmap('cool')

#sc=plt.scatter(TrueAs,resultAs,c=BInf.UVslope, vmin=-3, vmax=0, cmap=cm)

#plt.colorbar(sc,label="Slope")
#plt.plot([0.4,0.9],[0.4,0.9])
#plt.xlabel("True A")
#plt.ylabel("Resulting A")
#plt.show()

In [ ]:
cm = plt.cm.get_cmap('cool_r')



plt.show()

#cm = plt.cm.get_cmap('cool')

#sc=plt.scatter(TrueWs,resultWs,c=BInf.UVslope, vmin=-3, vmax=0, cmap=cm)
#plt.colorbar(sc,label="Slope")
#plt.plot([20,60],[20,60])
#plt.xlabel("True W0")
#plt.ylabel("Resulting W0")
#plt.show()

In [ ]:
down

In [ ]:
np.array(TrueWs)/np.array(resultWs)